In [ ]:
from google.colab import drive
import os
import zipfile

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Copy and Unzip (This is much faster than training directly from Drive)
zip_path = '/content/drive/MyDrive/Classes/AML/asos_images_raw.zip'
extract_path = '/content/dataset'

print("Unzipping data... this takes about 30-60 seconds...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Data ready!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unzipping data... this takes about 30-60 seconds...
Data ready!


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import concurrent.futures
from google.colab import drive

# --- 1. SETUP ---

# CONFIGURATION
BATCH_SIZE = 128
HIDDEN_DIM = 1024
NUM_LAYERS = 2
DROPOUT_RATE = 0.3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# PATHS
CLEAN_CSV_PATH = '/content/asos_final_clean.csv'
IMG_DIR = '/content/dataset/asos_images_raw'
MODEL_PATH = '/content/drive/MyDrive/Classes/AML/asos_final_best.pth'

# Check if data exists
if not os.path.exists(IMG_DIR):
    print("Images not found.")

# --- 2. DEFINE CLASSES ---

class ASOSMemDataset(Dataset):
    def __init__(self, dataframe, img_dir):
        self.metadata = dataframe.copy().reset_index(drop=True)
        self.img_dir = img_dir
        self.images = [None] * len(self.metadata)
        self.prices = [None] * len(self.metadata)

        print(f"Caching {len(self.metadata)} images...")
        def load_single_image(idx):
            try:
                row = self.metadata.iloc[idx]
                sku = str(int(row['sku']))
                img_path = os.path.join(self.img_dir, f"{sku}.jpg")
                with Image.open(img_path) as img:
                    return idx, img.convert('RGB').resize((224, 224)), float(row['price'])
            except:
                return idx, None, None

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(load_single_image, i) for i in range(len(self.metadata))]
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), unit="img"):
                idx, img, price = future.result()
                if img is not None:
                    self.images[idx] = img
                    self.prices[idx] = np.log1p(price)

        self.images = [img for img in self.images if img is not None]
        self.prices = [p for p in self.prices if p is not None]

    def __len__(self): return len(self.images)
    def __getitem__(self, idx): return self.images[idx], torch.tensor(self.prices[idx], dtype=torch.float32)

class TransformedDataset(Dataset):
    def __init__(self, base_dataset, transform):
        self.base = base_dataset
        self.transform = transform
    def __len__(self): return len(self.base)
    def __getitem__(self, idx):
        img, price = self.base[idx]
        return self.transform(img), price

def build_model():
    model = models.convnext_tiny(weights=None) # Weights will be loaded from file
    n_inputs = model.classifier[2].in_features

    # Construct Funnel Architecture
    layers = []
    current_in = n_inputs
    for i in range(NUM_LAYERS):
        out_dim = int(HIDDEN_DIM / (2**i))
        layers.append(nn.Linear(current_in, out_dim))
        layers.append(nn.BatchNorm1d(out_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(DROPOUT_RATE))
        current_in = out_dim
    layers.append(nn.Linear(current_in, 1))

    model.classifier[2] = nn.Sequential(*layers)
    return model

# --- 3. PREPARE TEST DATA ---
print("--- PREPARING TEST SET ---")
# Validation Transform only
val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create Split
df = pd.read_csv(CLEAN_CSV_PATH, on_bad_lines='skip', engine='python')
df = df[df['price'] < 1000]
train_val_df, test_df = train_test_split(df, test_size=0.10, random_state=42)

print(f"Test Set Size: {len(test_df)}")

--- PREPARING TEST SET ---
Test Set Size: 2998


In [ ]:
#phase 1
import pandas as pd
import numpy as np

#phase 2
!pip install category_encoders
from category_encoders import TargetEncoder

#phase 3
!pip install -q sentence-transformers
import re
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

#phase 4
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
from category_encoders import TargetEncoder
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 1. SETUP & CLEANING ---
print(f"Processing Metadata for {len(train_val_df)} training and {len(test_df)} test items...")

# Clean text for both splits
for d in [train_val_df, test_df]:
    d['description_clean'] = (
        d['description']
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .fillna('')
    )

# --- 2. TARGET ENCODING (High Cardinality: 'brand') ---
te = TargetEncoder(cols=['brand'])
# Fit on the training price
train_brand_te = te.fit_transform(train_val_df['brand'], train_val_df['price'])
test_brand_te = te.transform(test_df['brand'])

# --- 3. ONE-HOT ENCODING (Low Cardinality) ---
low_cardinality_cols = ['item_category', 'main_material', 'fit_type', 'color_simple']

# Get dummies
train_cats = pd.get_dummies(train_val_df[low_cardinality_cols], drop_first=False)
test_cats = pd.get_dummies(test_df[low_cardinality_cols], drop_first=False)

# If 'Red' color exists in Train but not Test, Test needs a column of 0s.
# If 'Purple' exists in Test but not Train, drop it (model doesn't know it).
test_cats = test_cats.reindex(columns=train_cats.columns, fill_value=0)

# --- 4. ASSEMBLE METADATA MATRIX ---
# Concatenate Target Encoded Brand + One-Hot Columns
X_meta_train = pd.concat([train_brand_te, train_cats], axis=1)
X_meta_test = pd.concat([test_brand_te, test_cats], axis=1)

print(f"Metadata Shape: {X_meta_train.shape}")

# --- 5. TEXT PROCESSING (BERT) ---
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

print("Generating BERT Embeddings (Train)...")
X_train_bert = bert_model.encode(
    train_val_df['description_clean'].tolist(),
    batch_size=64,
    show_progress_bar=True
)

print("Generating BERT Embeddings (Test)...")
X_test_bert = bert_model.encode(
    test_df['description_clean'].tolist(),
    batch_size=64,
    show_progress_bar=True
)

# --- 6. FINAL COMBINATION FOR MLP ---
# Stack Metadata + BERT embeddings
X_train_meta_bert = np.hstack([X_meta_train.values, X_train_bert])
X_test_meta_bert = np.hstack([X_meta_test.values, X_test_bert])

# Prepare Targets
y_train = train_val_df['price']
y_test = test_df['price']

print(f"Final Input Shape: {X_train_meta_bert.shape}")
print("Ready for MLP training.")

Processing Metadata for 26973 training and 2998 test items...
Metadata Shape: (26973, 97)
Generating BERT Embeddings (Train)...


Batches:   0%|          | 0/422 [00:00<?, ?it/s]

Generating BERT Embeddings (Test)...


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Final Input Shape: (26973, 481)
Ready for MLP training.


In [ ]:
# ---------------------------------------------------------
# Verify alignment - metadata already matches train/test split
# ---------------------------------------------------------

print("=== Feature Alignment Check ===")
print(f"Metadata Train: {X_meta_train.shape}")
print(f"Metadata Test:  {X_meta_test.shape}")
print(f"BERT Train:     {X_train_bert.shape}")
print(f"BERT Test:      {X_test_bert.shape}")
print(f"Target Train:   {y_train.shape}")
print(f"Target Test:    {y_test.shape}")

# Verify row counts match
assert X_meta_train.shape[0] == X_train_bert.shape[0] == len(y_train), "Train set size mismatch!"
assert X_meta_test.shape[0] == X_test_bert.shape[0] == len(y_test), "Test set size mismatch!"

print("\n✓ All features properly aligned!")

=== Feature Alignment Check ===
Metadata Train: (26973, 97)
Metadata Test:  (2998, 97)
BERT Train:     (26973, 384)
BERT Test:      (2998, 384)
Target Train:   (26973,)
Target Test:    (2998,)

✓ All features properly aligned!


In [ ]:
def evaluate_model(name, model, X_tr, y_tr, X_te, y_te, scaler=None):
    """
    Trains and evaluates a model with optional feature scaling.

    Args:
        name: Model name for display
        model: Sklearn-compatible model
        X_tr, y_tr: Training data
        X_te, y_te: Test data
        scaler: Optional pre-fitted scaler (if None, no scaling applied)
    """
    # Scale if scaler provided
    if scaler is not None:
        X_tr = scaler.transform(X_tr)
        X_te = scaler.transform(X_te)

    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)

    mae = mean_absolute_error(y_te, y_pred)
    rmse = np.sqrt(mean_squared_error(y_te, y_pred))
    r2 = r2_score(y_te, y_pred)

    print(f"\n=== {name} ===")
    print(f"MAE:  {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²:   {r2:.4f}")

    return {
        "model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "trained_model": model
    }

results = []

# --- METADATA + BERT COMBINATION ---
print(f"Combined feature shape: {X_train_meta_bert.shape}")

# --- FEATURE SCALING (Critical for MLP) ---
# BERT embeddings are already normalized, but metadata features are not
scaler = StandardScaler()
scaler.fit(X_train_meta_bert)

# --- MLP MODEL ---
mlp_meta_bert = MLPRegressor(
    hidden_layer_sizes=(512, 256, 128),  # Deeper network for richer features
    activation='relu',
    solver='adam',
    alpha=0.001,  # L2 regularization to prevent overfitting
    batch_size=128,
    learning_rate='adaptive',
    learning_rate_init=0.001,
    max_iter=100,  # Increased iterations
    early_stopping=True,  # Stop when validation score stops improving
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42,
    verbose=True  # Show training progress
)

results.append(
    evaluate_model(
        "Metadata + BERT (MLP)",
        mlp_meta_bert,
        X_train_meta_bert, y_train.to_numpy(),
        X_test_meta_bert, y_test.to_numpy(),
        scaler=scaler
    )
)

print("\n" + "="*50)
print("Model training complete!")

Combined feature shape: (26973, 481)
Iteration 1, loss = 280.96268927
Validation score: 0.697221
Iteration 2, loss = 163.06248327
Validation score: 0.709356
Iteration 3, loss = 139.67247275
Validation score: 0.707945
Iteration 4, loss = 124.49622726
Validation score: 0.734641
Iteration 5, loss = 113.65365931
Validation score: 0.705208
Iteration 6, loss = 99.06658639
Validation score: 0.734252
Iteration 7, loss = 90.02434775
Validation score: 0.704703
Iteration 8, loss = 79.72268913
Validation score: 0.742085
Iteration 9, loss = 72.31653332
Validation score: 0.743918
Iteration 10, loss = 63.79524046
Validation score: 0.720316
Iteration 11, loss = 55.44129575
Validation score: 0.740130
Iteration 12, loss = 52.47075639
Validation score: 0.716682
Iteration 13, loss = 48.32003830
Validation score: 0.731426
Iteration 14, loss = 44.76095692
Validation score: 0.726830
Iteration 15, loss = 43.47073746
Validation score: 0.732760
Iteration 16, loss = 39.86050346
Validation score: 0.725725
Iterati

In [ ]:
# =============================================================================
# STACKED MODEL: CNN (Images) + MLP (Metadata + BERT)
# =============================================================================

print("\n" + "="*60)
print("GENERATING PREDICTIONS FROM BOTH MODELS")
print("="*60)

# --- 1. CNN PREDICTIONS (Images) ---
print("\n[1/2] Loading CNN and generating image-based predictions...")

# Load Test Data into RAM
raw_test = ASOSMemDataset(test_df, IMG_DIR)
test_loader = DataLoader(
    TransformedDataset(raw_test, val_transforms),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

# Load CNN Model
cnn_model = build_model().to(DEVICE)
if os.path.exists(MODEL_PATH):
    print(f"✓ Loading CNN weights from: {MODEL_PATH}")
    cnn_model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
else:
    raise FileNotFoundError(f"CNN model not found at {MODEL_PATH}")

cnn_model.eval()

# Generate CNN predictions
cnn_preds_log = []
actual_targets_log = []

with torch.no_grad():
    for images, prices in tqdm(test_loader, desc="CNN Inference"):
        images = images.to(DEVICE)

        with torch.amp.autocast('cuda'):
            outputs = cnn_model(images)

        cnn_preds_log.extend(outputs.squeeze().float().cpu().numpy())
        actual_targets_log.extend(prices.numpy())

# Convert to arrays and inverse log transform
cnn_preds_log = np.array(cnn_preds_log)
cnn_preds_dollar = np.expm1(np.clip(cnn_preds_log, 0, 10))
actual_targets_dollar = np.expm1(np.array(actual_targets_log))

print(f"✓ CNN predictions shape: {cnn_preds_dollar.shape}")

# --- 2. MLP PREDICTIONS (Metadata + BERT) ---
print("\n[2/2] Generating metadata+BERT predictions from MLP...")

# Get the trained MLP from results
mlp_model = results[-1]['trained_model']

# Scale and predict
X_test_scaled = scaler.transform(X_test_meta_bert)
mlp_preds_dollar = mlp_model.predict(X_test_scaled)

print(f"✓ MLP predictions shape: {mlp_preds_dollar.shape}")

# --- 3. VERIFY ALIGNMENT ---
print("\n--- Alignment Check ---")
assert len(cnn_preds_dollar) == len(mlp_preds_dollar) == len(actual_targets_dollar), \
    "Prediction arrays have mismatched lengths!"
print(f"✓ All arrays aligned: {len(actual_targets_dollar)} samples")

# --- 4. ENSEMBLE STRATEGIES ---
print("\n" + "="*60)
print("TESTING ENSEMBLE STRATEGIES")
print("="*60)

def evaluate_ensemble(name, predictions, actuals):
    """Helper to evaluate ensemble performance"""
    mae = mean_absolute_error(actuals, predictions)
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    r2 = r2_score(actuals, predictions)

    print(f"\n{name}")
    print(f"  MAE:  £{mae:.2f}")
    print(f"  RMSE: £{rmse:.2f}")
    print(f"  R²:   {r2:.4f}")

    return {"name": name, "MAE": mae, "RMSE": rmse, "R2": r2}

ensemble_results = []

# Strategy 1: Simple Average
ensemble_avg = (cnn_preds_dollar + mlp_preds_dollar) / 2
ensemble_results.append(
    evaluate_ensemble("1. Simple Average (50/50)", ensemble_avg, actual_targets_dollar)
)

# Strategy 2: Weighted Average (try different weights)
weights_to_test = [
    (0.7, 0.3, "CNN-Heavy (70/30)"),
    (0.6, 0.4, "CNN-Leaning (60/40)"),
    (0.5, 0.5, "Balanced (50/50)"),  # Same as average
    (0.4, 0.6, "MLP-Leaning (40/60)"),
    (0.3, 0.7, "MLP-Heavy (30/70)")
]

for w_cnn, w_mlp, label in weights_to_test:
    ensemble_weighted = w_cnn * cnn_preds_dollar + w_mlp * mlp_preds_dollar
    ensemble_results.append(
        evaluate_ensemble(f"2. Weighted: {label}", ensemble_weighted, actual_targets_dollar)
    )

# Strategy 3: Learned Meta-Model (Ridge Regression)
print("\n3. Training Meta-Model (Ridge Stacking)...")

# Stack predictions as features
stacked_features = np.column_stack([cnn_preds_dollar, mlp_preds_dollar])

# Use Ridge to learn optimal weights
from sklearn.linear_model import Ridge
meta_model = Ridge(alpha=1.0)
meta_model.fit(stacked_features, actual_targets_dollar)
ensemble_meta = meta_model.predict(stacked_features)

learned_weights = meta_model.coef_
print(f"  Learned weights: CNN={learned_weights[0]:.3f}, MLP={learned_weights[1]:.3f}")
print(f"  Intercept: {meta_model.intercept_:.2f}")

ensemble_results.append(
    evaluate_ensemble("3. Meta-Model (Ridge)", ensemble_meta, actual_targets_dollar)
)

# --- 5. COMPARE WITH INDIVIDUAL MODELS ---
print("\n" + "="*60)
print("COMPARISON: Individual vs Ensemble")
print("="*60)

# Individual model performance
ensemble_results.insert(0,
    evaluate_ensemble("Baseline: CNN Only", cnn_preds_dollar, actual_targets_dollar)
)
ensemble_results.insert(1,
    evaluate_ensemble("Baseline: MLP Only", mlp_preds_dollar, actual_targets_dollar)
)

# --- 6. FIND BEST MODEL ---
print("\n" + "="*60)
print("FINAL RANKING (by RMSE)")
print("="*60)

ensemble_df = pd.DataFrame(ensemble_results).sort_values('RMSE')
print(ensemble_df.to_string(index=False))

best_model = ensemble_df.iloc[0]
print(f"\n🏆 BEST MODEL: {best_model['name']}")
print(f"   RMSE: £{best_model['RMSE']:.2f}")
print(f"   MAE:  £{best_model['MAE']:.2f}")
print(f"   R²:   {best_model['R2']:.4f}")


GENERATING PREDICTIONS FROM BOTH MODELS

[1/2] Loading CNN and generating image-based predictions...
Caching 2998 images...


100%|██████████| 2998/2998 [02:43<00:00, 18.39img/s]


✓ Loading CNN weights from: /content/drive/MyDrive/Classes/AML/asos_final_best.pth


CNN Inference: 100%|██████████| 24/24 [00:02<00:00,  8.31it/s]


✓ CNN predictions shape: (2998,)

[2/2] Generating metadata+BERT predictions from MLP...
✓ MLP predictions shape: (2998,)

--- Alignment Check ---
✓ All arrays aligned: 2998 samples

TESTING ENSEMBLE STRATEGIES

1. Simple Average (50/50)
  MAE:  £11.61
  RMSE: £20.38
  R²:   0.6540

2. Weighted: CNN-Heavy (70/30)
  MAE:  £13.22
  RMSE: £23.61
  R²:   0.5356

2. Weighted: CNN-Leaning (60/40)
  MAE:  £12.34
  RMSE: £21.89
  R²:   0.6007

2. Weighted: Balanced (50/50)
  MAE:  £11.61
  RMSE: £20.38
  R²:   0.6540

2. Weighted: MLP-Leaning (40/60)
  MAE:  £11.04
  RMSE: £19.12
  R²:   0.6954

2. Weighted: MLP-Heavy (30/70)
  MAE:  £10.63
  RMSE: £18.17
  R²:   0.7249

3. Training Meta-Model (Ridge Stacking)...
  Learned weights: CNN=0.266, MLP=0.886
  Intercept: -3.45

3. Meta-Model (Ridge)
  MAE:  £10.40
  RMSE: £17.09
  R²:   0.7567

COMPARISON: Individual vs Ensemble

Baseline: CNN Only
  MAE:  £16.37
  RMSE: £29.63
  R²:   0.2687

Baseline: MLP Only
  MAE:  £10.60
  RMSE: £17.59
  R²:  